In [2]:
import gonzag as gz
from gonzag.config import ldebug


In [3]:
file_sat = 'zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika'
name_ssh_sat = 'adt_unfiltered'
file_mod = 'zarr_ssh_ORCA1_20170101_20171231_grid_T'
name_ssh_mod = 'ssh'
file_lsm_mod = 'zarr_ssh_ORCA1_20170101_20171231_grid_T'
name_lsm_mod = '_FillValue'

In [4]:
(it1,it2), (Nts,Ntm) = gz.GetEpochTimeOverlap( file_sat , file_mod )

iszarr =  True
import zarr
 *** [GetTimeInfo()] Getting calendar/time info in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika ...
   => time/record dimension is "time"
 *** [GetTimeInfo()] Getting calendar/time info in zarr_ssh_ORCA1_20170101_20171231_grid_T ...
   => time/record dimension is "time_counter"

 *** [GetEpochTimeOverlap()] Earliest/latest dates:
   => for satellite data: 2017-04-01T23:57:40.480926208 2017-04-02T23:14:42.618708992 
   => for model     data: 2017-01-16T12:00:00.000000000 2017-12-16T12:00:00.000000000 



In [5]:
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2', it1,'--',it2)
    print('   => UTC: "'+str(it1)+'" to "'+str(it2)+'"\n')


    print('\n\n\n #####   M O D E L   2 D + T   D O M A I N   a.k.a.  S O U R C E   #####\n')



 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2017-04-01T23:57:40.480926208 -- 2017-04-02T23:14:42.618708992
   => UTC: "2017-04-01T23:57:40.480926208" to "2017-04-02T23:14:42.618708992"




 #####   M O D E L   2 D + T   D O M A I N   a.k.a.  S O U R C E   #####



In [6]:
clsm = name_lsm_mod
if name_lsm_mod=='_FillValue': clsm = name_lsm_mod+'@'+name_ssh_mod

In [7]:
ModelGrid = gz.ModGrid( file_mod, it1, it2, file_lsm_mod, clsm )

 *** [GetTimeEpochVector()] reading "time_counter" in zarr_ssh_ORCA1_20170101_20171231_grid_T and converting it to Epoch time...
 *** [GetTimeEpochVector()] reading "time_counter" in zarr_ssh_ORCA1_20170101_20171231_grid_T and converting it to Epoch time...
 *** Model latitude var is: "nav_lat" ! with 2 dimensions! (292, 362) 

 *** Model longitude var is: "nav_lon" ! with 2 dimensions! (292, 362) 


 *** what we use to define model land-sea mask:
    => "_FillValue@ssh" in "zarr_ssh_ORCA1_20170101_20171231_grid_T"

 *** [GridResolution()] Based on the longitude array, the model resolution ~=  1.0  degrees 

 *** Computing angle distortion of the model grid...

 *** About model gridded (source) domain:
     * shape =  (292, 362)
     * horizontal resolution:  1.0  degrees or  111.11  km
     * lon_min, lon_max =  0.0 360.0
     * Is this a global domain w.r.t longitude:  True
       ==> East West periodicity:  True , with an overlap of  2  points
     * lat_min, lat_max =  -78.58 89.74

In [8]:
    print('\n\n\n #####   S A T E L L I T E   1 D   T R A C K   a.k.a.  T A R G E T   #####\n')

    SatelliteTrack = gz.SatTrack( file_sat, it1, it2, Np=Nts, \
                                  domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )





 #####   S A T E L L I T E   1 D   T R A C K   a.k.a.  T A R G E T   #####

 *** [SatTrack()] Analyzing the time vector in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika ...
 *** [GetTimeEpochVector()] reading "time" in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika and converting it to Epoch time...
 *** [GetSatCoord()] reading "latitude" in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika ...
   => 44533 records in TOTAL!

 *** [GetSatCoord()] reading "longitude" in zarr_dt_global_alg_sla_vxxc_20170402_SARAL-Altika ...
   => 44533 records in TOTAL!


 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  44533
       ==> time record indices: 0 to 44532, included



In [9]:
MG=ModelGrid
ST=SatelliteTrack
file_out='mod2sat.nc'

In [12]:
import time ; # to report execution speed of certain parts of the code...
#
from gonzag.config import IsZarr, ldebug, if_talk, l_plot_meshes, deg2km, rfactor, search_box_w_km, l_save_track_on_model_grid, l_plot_meshes, rmissval
from gonzag.utils  import *
from gonzag.bilin_mapping import BilinTrack

In [13]:
    if not IsZarr:
        from gonzag.ncio   import GetModel2DVar, Save2Dfield, GetSatSSH, SaveTimeSeries
    else:
        from gonzag.zarrio import GetModel2DVar, Save2Dfield, GetSatSSH, SaveTimeSeries

In [14]:
(Nj,Ni) = MG.shape

In [15]:
print(Nj,Ni)

292 362


In [16]:
    d_found_km = rfactor*MG.HResDeg*deg2km
    print(' *** "found" distance criterion when searching for nearest point on model grid is ', d_found_km, ' km\n')

 *** "found" distance criterion when searching for nearest point on model grid is  83.3325  km



In [17]:
    # Size of the search zoom box:
    np_box_radius = SearchBoxSize( MG.HResDeg*deg2km, search_box_w_km )
    print(' *** Will use zoom boxes of width of '+str(2*np_box_radius+1)+' points for 1st attempts of nearest-point location...\n')

    Nt = ST.size ; # number of satellit observation point to work with here...


 *** Will use zoom boxes of width of 5 points for 1st attempts of nearest-point location...



In [18]:
   startTime = time.time()


In [19]:
print(startTime)

1617111721.057269


In [20]:
import sys
#from os import path, getcwd, mkdir
#import numpy as nmp

from math import radians, cos, sin, asin, sqrt, pi, tan, log, atan2, copysign
#from geopy import distance as geopy_distance

import numpy as nmp

from gonzag.config  import ldebug
from gonzag.utils   import Haversine, find_j_i_min, degE_to_degWE, degE_to_degWE_vctr


In [21]:
Yt=ST.lat
Xt=ST.lon
Ys=MG.lat
Xs=MG.lon
src_grid_local_angle=MG.xangle
k_ew_per=MG.EWPer
rd_found_km=d_found_km
np_box_r=np_box_radius
freq_talk=if_talk


In [22]:
sangle = src_grid_local_angle
kewp   = k_ew_per
rfound = rd_found_km
nprad  = np_box_r
(Nj,Ni) = Ys.shape

In [23]:
print(Nj,Ni)

292 362


In [25]:
NP = nmp.zeros((Nt,2)  , dtype=nmp.int64)

In [26]:
SM = nmp.zeros((Nt,4,2), dtype=nmp.int64)

In [27]:
WB = nmp.zeros((Nt,4))

In [28]:
ftalk = 2*Nt

In [29]:
ftalk = int(freq_talk)

In [30]:
        print('\n *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r =',rd_found_km, np_box_r,')')



 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 83.3325 2 )


In [35]:
    def nrpt(  ):
        #
        xnp = nmp.zeros((Nt,2), dtype=nmp.int64)
        #
        [jj,ji] = [nprad,nprad] ; # stupid first guess here...
        #
        for jt in range(Nt):
            ltalk = ((jt+1)%ftalk==0)

            if ltalk: print('      +++ Treated point: '+str(jt+1)+'/'+str(Nt), \
                            '\n          ==> Sat. coordinates:    ', round(Yt[jt],3), round(Xt[jt],3))
            
            [jj,ji] = NearestPoint( (Yt[jt],Xt[jt]), Ys, Xs, \
                                    rd_found_km=rfound, j_prv=jj, i_prv=ji, np_box_r=nprad )

            # Exclude points at first and last column if no E-W periodicity:
            if ji==0 or ji==Ni-1 and kewp == -1: [jj,ji] = [-1,-1]
            # Exclude points at first and last row:
            if jj==0 or jj==Nj-1:                     [jj,ji] = [-1,-1]
            
            xnp[jt,:] = [jj,ji]

            if ltalk: print('          ==> Model nearest point: ', \
                            round(Ys[jj,ji],3),round(Xs[jj,ji]%360.,3),' (',jj,ji,')')
        #
        return xnp


In [41]:
[jj,ji] = [nprad,nprad]
pcoor_trg=(Yt[0],Xt[0])
rd_found_km=rfound
j_prv=jj
i_prv=ji
np_box_r=nprad

In [42]:
(yT,xT) = pcoor_trg
(Ny,Nx) = Ys.shape

In [43]:
    jy, jx = -1,-1 ; # Flag value...
    j1=max(j_prv-np_box_r,0) ; j2=min(j_prv+np_box_r+1,Ny)
    i1=max(i_prv-np_box_r,0) ; i2=min(i_prv+np_box_r+1,Nx)
    lfound = False
    rfnd = rd_found_km
    igo = 0


In [46]:
        igo = igo + 1
        if igo>1: (j1,i1 , j2,i2) = (0,0 , Ny,Nx) ; # Falling back on whole domain for second pass...


In [53]:
print(Xs[j1:j2,i1:i2])

[[72.5 73.5 74.5 75.5 76.5]
 [72.5 73.5 74.5 75.5 76.5]
 [72.5 73.5 74.5 75.5 76.5]
 [72.5 73.5 74.5 75.5 76.5]
 [72.5 73.5 74.5 75.5 76.5]]


In [54]:
xd = Haversine( yT.values, xT.values,  Ys[j1:j2,i1:i2], Xs[j1:j2,i1:i2] )

In [37]:
def NearestPoint( pcoor_trg, Ys, Xs, rd_found_km=100., j_prv=0, i_prv=0, np_box_r=10 ):
    '''
    # * pcoor_trg : GPS coordinates (lat,lon) of target point    ([real],[real])
    # * Ys        : array of source grid latitude            2D numpy.array [real]
    # * Xs        : array of source grid longitude           2D numpy.array [real]
    '''
    (yT,xT) = pcoor_trg
    (Ny,Nx) = Ys.shape
    #
    jy, jx = -1,-1 ; # Flag value...
    j1=max(j_prv-np_box_r,0) ; j2=min(j_prv+np_box_r+1,Ny)
    i1=max(i_prv-np_box_r,0) ; i2=min(i_prv+np_box_r+1,Nx)
    lfound = False
    rfnd = rd_found_km
    igo = 0
    while (not lfound) and igo<5 :
        igo = igo + 1
        if igo>1: (j1,i1 , j2,i2) = (0,0 , Ny,Nx) ; # Falling back on whole domain for second pass...
        xd = Haversine( yT, xT,  Ys[j1:j2,i1:i2], Xs[j1:j2,i1:i2] )
        jy, jx = find_j_i_min( xd )
        lfound = ( xd[jy,jx] < rfnd )
        if igo>1 and not lfound:
            rfnd = 1.25*rfnd ; # increasing validation distance criterion by 25 %
    if igo==1:
        jy, jx = jy+j1, jx+i1 ; # found in the zoom box => translate to indices in whole domain:
    #
    if jy<0 or jx<0 or jy>=Ny or jx>=Nx or igo==5:
        print(' WARNING: did not find a nearest point for target point ',yT,xT,' !')
        print('          => last tested distance criterions =', rfnd,' km\n')
        jy, jx = -1,-1        
    return [jy,jx]

In [38]:
NP = nrpt( )

ValueError: applied function returned data with unexpected number of dimensions. Received 2 dimension(s) but expected 0 dimensions with names: ()